In [0]:
import zipfile
import shutil
import os, glob
import pandas as pd
import urllib.parse
import subprocess, sys
import py7zr
from pyspark.sql import functions as F
from pyspark.sql.functions import concat, concat_ws, lit, col, trim

In [0]:
pip install py7zr

Python interpreter will be restarted.
Collecting py7zr
 Downloading py7zr-0.16.2-py3-none-any.whl (66 kB)
Collecting texttable
 Downloading texttable-1.6.4-py2.py3-none-any.whl (10 kB)
Collecting pyppmd>=0.17.0
 Downloading pyppmd-0.17.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (127 kB)
Collecting brotli>=1.0.9; platform_python_implementation == "CPython"
 Downloading Brotli-1.0.9-cp38-cp38-manylinux1_x86_64.whl (357 kB)
Collecting multivolumefile>=0.2.3
 Downloading multivolumefile-0.2.3-py3-none-any.whl (17 kB)
Collecting pyzstd>=0.14.4
 Downloading pyzstd-0.15.0-cp38-cp38-manylinux2014_x86_64.whl (2.9 MB)
Collecting pycryptodomex>=3.6.6
 Downloading pycryptodomex-3.11.0-cp35-abi3-manylinux2010_x86_64.whl (1.9 MB)
Collecting pybcj>=0.5.0; platform_python_implementation == "CPython"
 Downloading pybcj-0.5.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (48 kB)
Installing collected packages: texttable, pyppmd, brotli, multivolumefile, pyzstd, pycryptodomex, pybcj, py7zr
Successfully installed brotli-1.0.9 multivolumefile-0.2.3 py7zr-0.16.2 pybcj-0.5.0 pycryptodomex-3.11.0 pyppmd-0.17.3 pyzstd-0.15.0 texttable-1.6.4
WARNING: You are using pip version 20.2.4; however, version 21.3.1 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-1e3e1e58-7172-434d-b33a-2a133509888b/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
dir_path = '/mnt/files/'
src_files = []
def walk_dir(dir_path):
  dir_files = dbutils.fs.ls(dir_path)
  for file in dir_files:
    if file.isDir():
      walk_dir(file.path)
    else: src_files.append(file.path)

In [0]:
walk_dir('/mnt/files/ir/files/Bookings/Consolidated/')
src_files

Out[3]: ['dbfs:/mnt/sources/6051a7a4b3c52f3187ce6a54/6051a8deb3c52f55a9ce6a55/AS and TS Data.csv',
 'dbfs:/mnt/sources/6051ae51b3c52fa1c8ce6a56/6051ae5cb3c52f250ece6a57/Products Bookings.csv',
 'dbfs:/mnt/sources/6051b3beb3c52f419ace6a58/6051b3c8b3c52f10dace6a59/Products Revenue.csv',
 'dbfs:/mnt/sources/6051b3beb3c52f419ace6a58/6051be86b3c52f6c4ece6a5c/Products Revenue.csv',
 'dbfs:/mnt/sources/6051bd3fb3c52f89e8ce6a5a/6051bd44b3c52fda01ce6a5b/Lookup PIDs.csv',
 'dbfs:/mnt/sources/6051d46fb3c52f32e0ce6a5d/6051d479b3c52f2400ce6a5e/Consolidated Rev_COGS_Part1.csv',
 'dbfs:/mnt/sources/6051d46fb3c52f32e0ce6a5d/6051dc22b3c52f0734ce6a5f/Consolidated Rev_COGSPart2.csv',
 'dbfs:/mnt/sources/6051e281b3c52fd7eece6a60/6051e286b3c52f68e4ce6a61/Out Of Scope PIDs.csv',
 'dbfs:/mnt/sources/6051ec44b3c52f74aace6a62/6051ec48b3c52fde23ce6a63/Lookup PIDs.csv',
 'dbfs:/mnt/sources/60bf231db3c52faa42ce6a64/60bf2329b3c52fb264ce6a65/Consolidate AS_0706.csv',
 'dbfs:/mnt/sources/60bf23d7b3c52f2c45ce6a66/60bf23deb3c52ff15ace6a67/Consolidate TS_0706.csv',
 'dbfs:/mnt/sources/60bf5944b3c52f65edce6a68/60bf5949b3c52f8eedce6a69/Consolidated Computing Business Booking_0706.csv',
 'dbfs:/mnt/sources/60bf5a78b3c52f75f7ce6a6a/60bf5a7db3c52f3005ce6a6b/Consolidated Rev_COGS_0706.csv',
 'dbfs:/mnt/sources/60bf5a78b3c52f75f7ce6a6a/60bf6dbcb3c52f8addce6a6c/Part1.csv',
 'dbfs:/mnt/sources/60bf5a78b3c52f75f7ce6a6a/60bf77f1b3c52f4e89ce6a6d/Part2.csv',
 'dbfs:/mnt/sources/60cc4599708255ee347f57ff/60cc459b708255e44b7f5800/DealCustomerMapping.csv',
 'dbfs:/mnt/sources/60cc8126708255bc5b7f5802/60cc812a70825521007f5803/CiscoTBData.csv',
 'dbfs:/mnt/sources/60cc8126708255bc5b7f5802/60d2d4227082552ba17f5807/CiscoTBData_Qtr.csv',
 'dbfs:/mnt/sources/60cc8126708255bc5b7f5802/60d2e7cd70825515287f5808/CiscoTBData_Qtr.csv',
 'dbfs:/mnt/sources/60cc8126708255bc5b7f5802/60d2fb1970825525447f5809/CiscoTBData_Qtr.csv',
 'dbfs:/mnt/sources/60cc8126708255bc5b7f5802/60d5664b708255ba2d7f580e/CiscoTBData_Qtr.csv',
 'dbfs:/mnt/sources/60cc8126708255bc5b7f5802/6131d60bedfaf81337264027/CiscoTBData_Qtr_JUL21.csv',
 'dbfs:/mnt/sources/60cc8126708255bc5b7f5802/6131ffbdedfaf8697826402a/CiscoTBData_Qtr_JUL21.csv',
 'dbfs:/mnt/sources/60d9a0247082550eff7f5814/60d9a04870825500047f5815/Sheet2.csv',
 'dbfs:/mnt/sources/60d9a0247082550eff7f5814/60d9a04870825500047f5815/TMS Split.xlsx',
 'dbfs:/mnt/sources/60d9a3b970825508a07f5816/60d9a3c170825520f57f5817/PF and PID mapping.xlsx',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/60def326708255193d7f5823/Test JE 1.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/60def354708255576e7f5824/Test JE 2.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/60df41ae7082555abf7f5825/Test1.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/61018ead7082555bb37f5855/Test Entry.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/610294017082558a867f5869/Test JE 3.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/610299237082551c527f586a/Test JE 4.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/6102ad8d708255a8c57f586b/Test JE 5.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/6102bc6f70825514977f586c/Test JE 6.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/6102e21870825541e07f5876/Test Entry.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/615d53b3edfaf8154c264133/Test New 1.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/615d5439edfaf8606d264134/Test New 2.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/615d54c4edfaf80d7d264135/Test New 3.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/615d5b46edfaf83425264136/Test 5.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/615d5c6cedfaf8c621264137/Test New 6.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/616011dfedfaf885e926414f/Test New 7.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/616425caedfaf81079264150/Test 10112021.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/61642e4fedfaf831fc264152/Test 10112021_1.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/61711543323eaf2c7adff22e/Test 102

In [0]:
# Set parameter for folder
dbutils.widgets.text("source_url","UNAVAILABLE")

In [0]:
# Get zip file path mounted
source_dir = dbutils.widgets.get("source_url")
mounted_path_dir = urllib.parse.unquote(source_dir.split("?")[0])
source_file_path_dir = f"/dbfs/mnt/{mounted_path_dir}" #if pandas reading in the file
# source_file_path = f"dbfs:/mnt/{mounted_path}" #if koalas reading in the file
source_file_path_dir

for fp in glob.iglob(fr"{source_file_path_dir}/*.zip"):
  print(f"FROM: {fp}")
for fp in glob.iglob(fr"{source_file_path_dir}/*.7z"):
  print(f"FROM: {fp}")

FROM: /dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/As of 11_12_21/Enterprise Switching_201812,201903,201906.zip
FROM: /dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/As of 11_12_21/Enterprise Switching_FY201811_Complete.zip

In [0]:
%fs cp 'dbfs:/mnt/files/ir/files/Bookings/Collaboration/Collaboration_FY2015.zip' 'dbfs:/FileStore/tables/Carmel/TSS_BE_Zip/Collaboration_FY15.zip'

res10: Boolean = true

In [0]:
%fs cp 'dbfs:/mnt/files/ir/files/Bookings/Monitoring & Analysis/Monitoring and analysis_FY20.zip' 'dbfs:/FileStore/tables/Carmel/TSS_BE_Zip/Monitoring & Analysis_FY20.zip'

res1: Boolean = true

In [0]:
%fs cp 'dbfs:/mnt/files/ir/files/Bookings/Enterprise Switching/version 2/201808(week4)_Enterprise Switching.csv' 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip4/201808(week4)_Enterprise Switching.csv'

res0: Boolean = true

In [0]:
%fs mv 'dbfs:/FileStore/tables/Carmel/TSS_BE_Zip/Monitoring & Analysis_FY20.zip' 'dbfs:/FileStore/tables/Carmel/TSS_BE_Zip/Monitoring_Analysis_FY20.zip'

res2: Boolean = true

In [0]:
# Set parameter for zip file
dbutils.widgets.text("file_url","UNAVAILABLE")

In [0]:
file_path = dbutils.widgets.get("file_url")
mounted_path = urllib.parse.unquote(file_path.split("?")[0])
source_file_path = f"/dbfs/mnt/{mounted_path}" #if pandas reading in the file
# source_file_path = f"dbfs:/mnt/{mounted_path}" #if koalas reading in the file
source_file_path

Out[7]: '/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/As of 11_12_21/Enterprise Switching_FY201811_Complete.zip'

In [0]:
def traverse_dir_recur(dir):
    import os
    l = os.listdir(dir)
    for d in l:
        if os.path.isdir(dir + d):
            traverse_dir_recur(dir+  d +"/")
        else:
            file_path = dir + d
            print(file_path)

In [0]:
traverse_dir_recur("/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/") 

/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016011.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016012.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016013.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016014.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016021.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016022.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016023.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016024.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016031.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016032.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016033.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016034.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016035.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016041.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016042.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016043.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016044.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016051.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016052.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016053.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016054.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016061.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016062.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016063.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016064.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016065_SLS_ADJ.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016071.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016072.csv
/dbfs/mnt/files/ir/files/Bookings/Enterprise Switching/ES_16_18_19_csv/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise switching_2016073.csv
/dbfs/mnt/files/ir/

In [0]:
def main(zip_path): 
#     dest_dir = r"/dbfs/mnt/files/ir/files/Bookings/Collaboration/"
    unzip_dir= r"/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/"
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.printdir()
            zip_ref.extractall(unzip_dir)
            zip_ref.close()
#     for fp in glob.iglob(fr"{dest_dir}/*.csv"):
#         print(f"FROM: {fp}")
#         dest = os.path.join("dbfs:"+unzip_dir[5:],os.path.basename(fp))
#         print(f"TO: {dest}")
#         dbutils.fs.mv("dbfs:"+fp[5:], dest)
    except:
        print("An exception occurred extracting with Python ZipFile library.")
        print("Attempting to extract using 7zip")
        with py7zr.SevenZipFile(zip_path, 'r') as zip_ref:
#             zip_ref.printdir()
            zip_ref.extractall(unzip_dir)
#         subprocess.Popen(["7z", "e", f"{zip_path}", f"-o{unzip_dir}", "-y"])

In [0]:
# Run unzip function for each zip file in the folder
main(source_file_path)

File Name Modified Size
Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181101_ES_Inlist.csv 2021-11-11 13:09:10 136205598
Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181101_ES_Notinlist.csv 2021-11-11 13:16:02 167384821
Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181101_ES_Null.csv 2021-11-11 13:22:30 111476016
Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181102_ES_Inlist.csv 2021-11-11 13:28:04 193048628
Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181102_ES_Notinlist.csv 2021-11-11 13:31:48 166933490
Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181102_ES_Null.csv 2021-11-11 13:38:26 114531426
Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181103_ES_Inlist.csv 2021-11-11 13:56:36 172465510
Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181103_ES_Notinlist.csv 2021-11-11 15:05:56 173705524
Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181103_ES_Null.csv 2021-11-11 15:13:02 108987072
Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181104_ES_Inlist.csv 2021-11-11 15:24:48 212511766
Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181104_ES_Notinlist.csv 2021-11-11 15:29:36 182226356
Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_201841104_ES_Null.csv 2021-11-11 15:33:20 144299585

In [0]:
# Set parameter
dbutils.widgets.text("source_dir","UNAVAILABLE")

In [0]:
# Get zip file path mounted

for fp in glob.iglob(fr"{source_dir}/*.zip"):
  print(f"FROM: {fp}")
#   file_path = dbutils.widgets.get(fp)
  for zipfile in fp:
    print(zipfile)
    file_path = dbutils.widgets.get(zipfile[8:-1])
    mounted_path = urllib.parse.unquote(file_path.split("?")[0])
    source_file_path = f"/dbfs/mnt/{mounted_path}" #if pandas reading in the file
    # source_file_path = f"dbfs:/mnt/{mounted_path}" #if koalas reading in the file
    main(source_file_path)

In [0]:
%python
dir_path = '/mnt/files/ir/files/Bookings/ES_Unzip7/'
src_files = []
def walk_dir(dir_path):
  dir_files = dbutils.fs.ls(dir_path)
  for file in dir_files:
    if file.isDir():
      walk_dir(file.path)
    else: src_files.append(file.path)

In [0]:
%python
walk_dir('/mnt/files/ir/files/Bookings/ES_Unzip7/')
src_files

Out[10]: ['dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018121_Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018121_Non Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018122_Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018122_Non Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018123_Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018123_Non Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018124_Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018124_Non Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018125_Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018125_Non Americas_AR.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018125_Non Americas_ERP.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018125_Non Americas_Non ERP AR SLS Adj.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018125_Non Americas_SLS Adj.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019031_Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019031_Non Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019032_Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019032_Non Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019033_Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019033_Non Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019034_Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019034_Non Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019035_Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019035_Non Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019061_Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019061_Non Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019062_Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019062_Non Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019063_Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019063_Non Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019064_Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019064_Non Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019065_Americas_Non US.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019065_Americas_US.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019065_Non Americas.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181101_ES_Inlist.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181101_ES_Notinlist.csv',
 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181101_ES_Null.csv',
 'dbfs:/mnt/files/ir/files/Booking

In [0]:
# Define the source and destination path
source = "/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/"

# code to move the files from sub-folder to main folder.
def move_subfolder(dir):
    import shutil
    import os
    l = os.listdir(dir)
    destination = "/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/"
    for d in l:
        if os.path.isdir(dir + d):
            move_subfolder(dir+  d +"/")
        else:
            file_path = dir + d
            shutil.move(file_path, destination)
            print(file_path)
            

In [0]:
# def move_subfolder(dir):
#     import shutil
#     import os
#     l = os.listdir(dir)
#     destination = "/dbfs/mnt/files/ir/files/Bookings/ES_Unzip/"
#     for d in l:
#         if os.path.isdir(dir + d):
#             move_subfolder(dir+  d +"/")
#         else:
#             file_path = dir + d
#             if os.path.isfile(file_path)==False and os.access(file_path, os.R_OK):
#               shutil.move(file_path, destination)
#               print(file_path)
#             else:
#               continue

In [0]:
move_subfolder(source)

/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018121_Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018121_Non Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018122_Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018122_Non Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018123_Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018123_Non Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018124_Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018124_Non Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018125_Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018125_Non Americas_AR.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018125_Non Americas_ERP.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018125_Non Americas_Non ERP AR SLS Adj.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2018125_Non Americas_SLS Adj.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019031_Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019031_Non Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019032_Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019032_Non Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019033_Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019033_Non Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019034_Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019034_Non Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019035_Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019035_Non Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019061_Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019061_Non Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019062_Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019062_Non Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019063_Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019063_Non Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019064_Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019064_Non Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019065_Americas_Non US.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019065_Americas_US.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/ES-today/Enterprise Switching_2019065_Non Americas.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181101_ES_Inlist.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181101_ES_Notinlist.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181101_ES_Null.csv
/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_FY201811_Complete/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181102_ES_Inlist.csv
/dbfs/mnt/files/ir/files/Bookings/ES_

In [0]:
# df1r = spark.read.format("csv") \
#       .option("header", "true") \
#       .option("delimiter", ",") \
#       .option("inferSchema","true") \
#       .option("ESCAPE quote", '"') \
#       .option("encoding", "UTF-8")\
#       .load('dbfs:/mnt/files/ir/files/Bookings/Test_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201601.csv')

df1r = spark.read.format("csv") \
      .option("header", "true") \
      .option("delimiter", ",") \
      .option("inferSchema","true") \
      .option("quote", "\"") \
      .option("escape", "\"") \
      .option("encoding", "UTF-8")\
      .load('dbfs:/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2018125_Non Americas_SLS Adj.csv')

In [0]:
df1r.show()

+-----------+----------+--------------------+-----------------+----------------+---------------------------+----------------------------------+-------------------------+----------------------------+--------------------------+-------------------------+-----------------------+---------------------------+-------------------------+-----------------------+-----------------+------------------+--------------------+-------------------------+-----------------------+---------------------------------+---------------------+-------------------+----------------------+------------+-----------+--------------+-------------+-------------+-----------------+--------------+-----------------------+--------------------+
Fiscal Year|Product ID| PH-PID|Fiscal Quarter ID|Fiscal Period ID|Finance BU/Service Category|Finance Product Family/Allctd Svcs|Bookings Transaction Type|Bill To Global Ultimate Name|Bill To Global Ultimate ID|ERP Bill To Customer Name|ERP Bill To Customer Id|ERP Bill To Customer Number|ERP Sold To Customer Name|ERP Sold To Customer Id|Ship To Site City|Ship To Site State|Ship To Site Country|ERP Ship To Customer Name|ERP Ship To Customer Id|End Customer Global Ultimate Name|ERP End Customer Name|ERP End Customer Id|ERP Sales Order Number|Web Order ID|ERP Deal ID| Country|Sales Level 1|Sales Level 2| Sales Level 3| Sales Level 4|CA Service Bookings Net| TSS Methodology|
+-----------+----------+--------------------+-----------------+----------------+---------------------------+----------------------------------+-------------------------+----------------------------+--------------------------+-------------------------+-----------------------+---------------------------+-------------------------+-----------------------+-----------------+------------------+--------------------+-------------------------+-----------------------+---------------------------------+---------------------+-------------------+----------------------+------------+-----------+--------------+-------------+-------------+-----------------+--------------+-----------------------+--------------------+
 2018|CON-RCBR-1| 1200W POWER CABLES| 2018Q4| 201812| TECHNICAL SUPPORT...| SMART NET TOTAL CARE| SLS_ADJ| UNKNOWN| -999| UNKNOWN| -999| UNKNOWN| UNKNOWN| -999| null| null| UNKNOWN| UNKNOWN| -999| UNKNOWN| UNKNOWN| -999| 106815742| UNKNOWN| null|UNITED KINGDOM| EMEAR-REGION| EMEAR_OTHER| EMEAR_REBATES| EMEAR_CSPP| -23.90069| Country Allocation|
 2018|CON-RCBR-1| WS-C4506-E| 2018Q4| 201812| TECHNICAL SUPPORT...| SMART NET TOTAL CARE| SLS_ADJ| UNKNOWN| -999| UNKNOWN| -999| UNKNOWN| UNKNOWN| -999| null| null| UNKNOWN| UNKNOWN| -999| UNKNOWN| UNKNOWN| -999| 106571170| UNKNOWN| null| UNKNOWN| EMEAR-REGION| EMEAR_OTHER| EMEAR_REBATES| EMEAR_CSPP| 3.857052|Prior Period Orde...|
 2018|CON-RCBR-1|C9300L_Enterprise...| 2018Q4| 201812| TECHNICAL SUPPORT...| SMART NET TOTAL CARE| SLS_ADJ| UNKNOWN| -999| UNKNOWN| -999| UNKNOWN| UNKNOWN| -999| null| null| UNKNOWN| UNKNOWN| -999| UNKNOWN| UNKNOWN| -999| 106814452| UNKNOWN| null| UNKNOWN| APJC|GREATER_CHINA| GC_CONTRA| GC_CONTRA_OPS| -1.779037|20 dollar Range -...|
 2018|CON-RCBR-1| 1200W POWER CABLES| 2018Q4| 201812| TECHNICAL SUPPORT...| SMART NET TOTAL CARE| SLS_ADJ| UNKNOWN| -999| UNKNOWN| -999| UNKNOWN| UNKNOWN| -999| null| null| UNKNOWN| UNKNOWN| -999| UNKNOWN| UNKNOWN| -999| 106813640| UNKNOWN| 18690043| UNKNOWN| EMEAR-REGION| EMEAR_OTHER| EMEAR_REBATES| EMEAR_CSPP| -483.789232| Country Allocation|
 2018|CON-RCBR-1| SDA-DACH-BNDL| 2018Q4| 201812| TECHNICAL SUPPORT...| SMART NET TOTAL CARE| SLS_ADJ| UNKNOWN| -999| UNKNOWN| -999| UNKNOWN| UNKNOWN| -999| null| null| UNKNOWN| UNKNOWN| -999| UNKNOWN| UNKNOWN| -999| 106844743| UNKNOWN| 18828387| UNKNOWN| APJC|GREATER_CHINA| GC_CONTRA| GC_CONTRA_OPS| -13.215508| Country Allocation|
 2018|CON-RCBR-1| WS-C2960+24TCJ1-RF| 2018Q4| 201812| TECHNICAL SUPPORT...| SMART NET TOTAL CARE| SLS_ADJ| UNKNOWN| -999| UNKNOWN| -999| UNKNOWN| UNKNOWN| -999| null| null| UNKNOWN| UNKNOWN| -999| UNKNOWN| UNKNO

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window
from pyspark.sql import functions as F

# Check if csv delimiter read correctly in spark dataframe
df1rr = df1r.withColumn("RN", row_number().over(Window.orderBy(monotonically_increasing_id()))-1)

print(df1rr.select("CA Service Bookings Net", F.col("CA Service Bookings Net").cast('float').isNotNull().alias("IfNumber")).filter("IfNumber == false").count())
df1rr.select("RN", "CA Service Bookings Net", F.col("CA Service Bookings Net").cast('float').isNotNull().alias("IfNumber")).filter("IfNumber == false").show()

0
+---+-----------------------+--------+
 RN|CA Service Bookings Net|IfNumber|
+---+-----------------------+--------+
+---+-----------------------+--------+

In [0]:
%fs rm -r 'dbfs:/mnt/files/ir/files/Bookings/ES_Unzip/Enterprise switching_FY2019.zip'

res2: Boolean = true

In [0]:
### Add missing columns as None in dataframe before concating - multiple dataframes

from functools import reduce
from pyspark.sql import DataFrame
import pyspark.sql.functions as F

def unionAll(*dfs, fill_by=None):
    clmns = {clm.name.lower(): (clm.dataType, clm.name) for df in dfs for clm in df.schema.fields}
    
    dfs = list(dfs)
    for i, df in enumerate(dfs):
        df_clmns = [clm.lower() for clm in df.columns]
        for clm, (dataType, name) in clmns.items():
            if clm not in df_clmns:
                # Add the missing column
                dfs[i] = dfs[i].withColumn(name, F.lit(fill_by).cast(dataType))
    return reduce(DataFrame.unionByName, dfs)


# show pyspark dataframe in pandas format
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)


In [0]:
# Set parameter
dbutils.widgets.text("unzip_dir","UNAVAILABLE")

In [0]:

# Get zip file path mounted
unzip_dir = dbutils.widgets.get("unzip_dir")
mounted_path_dir = urllib.parse.unquote(unzip_dir.split("?")[0])
unzip_file_path_dir = f"/dbfs/mnt/{mounted_path_dir}" #if pandas reading in the file
# source_file_path = f"dbfs:/mnt/{mounted_path}" #if koalas reading in the file
print(unzip_file_path_dir)

for fp in glob.iglob(fr"{unzip_file_path_dir}/*.csv"):
  print(f"FROM: {fp}")


/dbfs/mnt/files/ir/files/Bookings/ER_Unzip/
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_201903_Americas.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_201903_Non-Americas.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_201904.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019051&2019052.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019053&2019054.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019061&2019062.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019063&2019064.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019065.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019071_2019072.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019073&2019074.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019081&2019082.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019083&2019084.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019091&2019092.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019093.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019094.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019095.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019101&2019102.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019103&2019104.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019111.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019112.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019113&2019114.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_20191201&20191202.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_20191203.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2019124&2019125.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2020011.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2020012.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2020013.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2020014.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2020021.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2020022.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2020023.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2020024.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise Routing_2020031.csv
FROM: /dbfs/mnt/files/ir/files/Bookings/ER_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_Enterprise 

In [0]:
# Set up schema of dataframe
from pyspark.sql.types import StructType

schema = [i for i in df1r.schema] 

NewSchema = StructType(schema)

NewSchema

Out[16]: StructType(List(StructField(Fiscal Year,IntegerType,true),StructField(Product ID,StringType,true),StructField(PH-PID,StringType,true),StructField(Fiscal Quarter ID,StringType,true),StructField(Fiscal Period ID,IntegerType,true),StructField(Finance BU/Service Category,StringType,true),StructField(Finance Product Family/Allctd Svcs,StringType,true),StructField(Bookings Transaction Type,StringType,true),StructField(Bill To Global Ultimate Name,StringType,true),StructField(Bill To Global Ultimate ID,IntegerType,true),StructField(ERP Bill To Customer Name,StringType,true),StructField(ERP Bill To Customer Id,IntegerType,true),StructField(ERP Bill To Customer Number,StringType,true),StructField(ERP Sold To Customer Name,StringType,true),StructField(ERP Sold To Customer Id,IntegerType,true),StructField(Ship To Site City,StringType,true),StructField(Ship To Site State,StringType,true),StructField(Ship To Site Country,StringType,true),StructField(ERP Ship To Customer Name,StringType,true),StructField(ERP Ship To Customer Id,IntegerType,true),StructField(End Customer Global Ultimate Name,StringType,true),StructField(ERP End Customer Name,StringType,true),StructField(ERP End Customer Id,IntegerType,true),StructField(ERP Sales Order Number,IntegerType,true),StructField(Web Order ID,StringType,true),StructField(ERP Deal ID,IntegerType,true),StructField(Country,StringType,true),StructField(Sales Level 1,StringType,true),StructField(Sales Level 2,StringType,true),StructField(Sales Level 3,StringType,true),StructField(Sales Level 4,StringType,true),StructField(CA Service Bookings Net,DoubleType,true),StructField(TSS Methodology,StringType,true)))

In [0]:
from pyspark.sql import DataFrame

def Union_all_df(dir):
    import os
    l = os.listdir(dir)
    union_df = spark.createDataFrame([], NewSchema)
#     # Check if union_df exists
#     if union_df is not None and isinstance(union_df,DataFrame):
#       union_df = union_df
#     else:
#       union_df = spark.createDataFrame([], NewSchema)
      
      #  Loop over files in the directory/sub directory
    for d in l:
        if os.path.isdir(dir + d):
            Union_all_df(dir +  d +"/")
        else:
            file_path = dir + d
            print(file_path[5:])
            df = spark.read.format("csv") \
                             .option("header", "true") \
                             .option("delimiter", ",") \
                             .option("inferSchema","true") \
                             .option("quote", "\"") \
                             .option("escape", "\"") \
                             .option("encoding", "UTF-8")\
                             .load(file_path[5:])

            # Union all dfs
            union_df = unionAll(union_df, df)
            print(union_df.count()) # print num of rows unioned
    return union_df


In [0]:
union_df = Union_all_df("/dbfs/mnt/files/ir/files/Bookings/ES_Unzip7/")

/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2018121_Americas.csv
407296
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2018121_Non Americas.csv
1156318
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2018122_Americas.csv
1454423
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2018122_Non Americas.csv
2089672
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2018123_Americas.csv
2565026
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2018123_Non Americas.csv
3289778
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2018124_Americas.csv
3838014
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2018124_Non Americas.csv
4654208
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2018125_Americas.csv
5493584
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2018125_Non Americas_AR.csv
5511635
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2018125_Non Americas_ERP.csv
5589780
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2018125_Non Americas_Non ERP AR SLS Adj.csv
6633349
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2018125_Non Americas_SLS Adj.csv
6777310
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019031_Americas.csv
7140583
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019031_Non Americas.csv
7820549
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019032_Americas.csv
8221741
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019032_Non Americas.csv
8911094
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019033_Americas.csv
9259524
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019033_Non Americas.csv
9887379
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019034_Americas.csv
10265967
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019034_Non Americas.csv
10959075
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019035_Americas.csv
11438432
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019035_Non Americas.csv
12385797
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019061_Americas.csv
12627445
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019061_Non Americas.csv
13100711
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019062_Americas.csv
13424237
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019062_Non Americas.csv
13937831
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019063_Americas.csv
14560051
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019063_Non Americas.csv
15265045
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019064_Americas.csv
15739573
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019064_Non Americas.csv
16515782
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019065_Americas_Non US.csv
17179921
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019065_Americas_US.csv
17661300
/mnt/files/ir/files/Bookings/ES_Unzip7/Enterprise Switching_2019065_Non Americas.csv
18630652
/mnt/files/ir/files/Bookings/ES_Unzip7/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181101_ES_Inlist.csv
18892860
/mnt/files/ir/files/Bookings/ES_Unzip7/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181101_ES_Notinlist.csv
19229348
/mnt/files/ir/files/Bookings/ES_Unzip7/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181101_ES_Null.csv
19487308
/mnt/files/ir/files/Bookings/ES_Unzip7/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181102_ES_Inlist.csv
19860858
/mnt/files/ir/files/Bookings/ES_Unzip7/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181102_ES_Notinlist.csv
20194657
/mnt/files/ir/files/Bookings/ES_Unzip7/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181102_ES_Null.csv
20459567
/mnt/files/ir/files/Bookings/ES_Unzip7/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___14588525_20181103_ES_Inlist.csv
20789628
/mnt/file

In [0]:
union_df.show()

+-----------+--------------+------------------+-----------------+----------------+---------------------------+----------------------------------+-------------------------+----------------------------+--------------------------+-------------------------+-----------------------+---------------------------+-------------------------+-----------------------+-----------------+------------------+--------------------+-------------------------+-----------------------+---------------------------------+---------------------+-------------------+----------------------+------------+-----------+-------------+-------------+--------------------+--------------------+--------------------+-----------------------+--------------------+
Fiscal Year| Product ID| PH-PID|Fiscal Quarter ID|Fiscal Period ID|Finance BU/Service Category|Finance Product Family/Allctd Svcs|Bookings Transaction Type|Bill To Global Ultimate Name|Bill To Global Ultimate ID|ERP Bill To Customer Name|ERP Bill To Customer Id|ERP Bill To Customer Number|ERP Sold To Customer Name|ERP Sold To Customer Id|Ship To Site City|Ship To Site State|Ship To Site Country|ERP Ship To Customer Name|ERP Ship To Customer Id|End Customer Global Ultimate Name|ERP End Customer Name|ERP End Customer Id|ERP Sales Order Number|Web Order ID|ERP Deal ID| Country|Sales Level 1| Sales Level 2| Sales Level 3| Sales Level 4|CA Service Bookings Net| TSS Methodology|
+-----------+--------------+------------------+-----------------+----------------+---------------------------+----------------------------------+-------------------------+----------------------------+--------------------------+-------------------------+-----------------------+---------------------------+-------------------------+-----------------------+-----------------+------------------+--------------------+-------------------------+-----------------------+---------------------------------+---------------------+-------------------+----------------------+------------+-----------+-------------+-------------+--------------------+--------------------+--------------------+-----------------------+--------------------+
 2018| CON-SNT-SMS-1| WS-C2960-48PST-L| 2018Q4| 201812| TECHNICAL SUPPORT...| SMART NET TOTAL CARE| POS | TERACAI CORPORATION| 142957889| TERACAI CORPORATION| 12915956| UNKNOWN| TERACAI CORPORATION| 12915956| SYRACUSE| NY| UNITED STATES| TERACAI CORPORATION| 12915956| WIELAND CHASE LLC| CHASE BRASS & COP...| 4967677| -9999| UNKNOWN| null|UNITED STATES| Americas| US COMMERCIAL|SBU_US COMMERCIAL...|SBU_USC CENTRAL S...| 0.124458|Country & SKU Lev...|
 2018| CON-RCBR-1|C2960P_SW_ADJ_PROD| 2018Q4| 201812| TECHNICAL SUPPORT...| SMART NET TOTAL CARE| SLS_ADJ| UNKNOWN| -999| UNKNOWN| -999| UNKNOWN| UNKNOWN| -999| null| null| UNKNOWN| UNKNOWN| -999| UNKNOWN| UNKNOWN| -999| 106664165| UNKNOWN| 18642681| BRAZIL| Americas| LATIN AMERICA| BRASIL| BRASIL HQ| -0.020062| Country Allocation|
 2018| SP-AR1-1|1200W POWER CABLES| 2018Q4| 201812| TECHNICAL SUPPORT...| SP PROGRAMS| ERP | CS CAPITAL| 5919419| CISCO SYSTEMS CAP...| 50295| 50650| CISCO SYSTEMS CAP...| 50295| RICHARDSON| TX| UNITED STATES| SBCSI PURCHASING ...| 1146647| AT&T INC| SBCSI PURCHASING ...| 1146647| 106638592| UNKNOWN| 17288907|UNITED STATES| Americas| AMERICAS_SP| ASP TELCO MOBILE| ASP ATT| -2.890451| Country Allocation|
 2018| CON-NCFT-1|WS-C4506E-S6L-96V+| 2018Q4| 201812| TECHNICAL SUPPORT...| AS CORE| ERP | RED RIVER COMPUTE...| 214108| RED RIVER COMPUTE...| 44503| 44475| RED RIVER COMPUTE...| 44503| MCCLELLAN| CA| UNITED STATES| DEPT OF HOMELAND ...| 990280| GOVERNMENT OF THE US| DEPT OF HOMELAND ...| 990280| 106568063| UNKNOWN| 17966779|UNITED STATES| Americas|US PS Market Segment| PS FEDERAL AREA|FED-NATIONAL SECU...| -6.25E-7|Prior Period Orde...|
 2018|CON-SNTP-SMS-1|WS-C4506E-S6L-96V+| 2018Q4| 201812| TECHNICAL SUPPORT...| SMART NET TOTAL CARE| POS | EPLUS INC| 221116| SYSTEMS MANAGEMEN...| 2087885| UNKNOWN| SYSTEMS MANAGEMEN...| 2087885| ROCHESTER| NY| UNITED STATES| SYSTEMS

In [0]:
union_df.groupBy('Fiscal Year').agg({'CA Service Bookings Net': 'sum'}).orderBy(F.col('Fiscal Year').asc()).show()
display(union_df.groupBy('Fiscal Year').agg({'CA Service Bookings Net': 'sum'}).orderBy(F.col('Fiscal Year').asc()).toPandas())

+-----------+----------------------------+
Fiscal Year|sum(CA Service Bookings Net)|
+-----------+----------------------------+
 2016| 2.2915341386378217E9|
 2018| 1.550850455159726E9|
 2019| 1.7006142403922198E9|
 2020| 2.3961754247120123E9|
 2021| 2.228126338616163E9|
+-----------+----------------------------+

Fiscal Year,sum(CA Service Bookings Net)
2016,2.2915341386378236E9
2018,1.5508504551597266E9
2019,1.700614240392217E9
2020,2.3961754247120204E9
2021,2.2281263386161695E9


In [0]:
display(union_df.groupBy('Fiscal Quarter ID').agg({'CA Service Bookings Net': 'sum'}).orderBy(F.col('Fiscal Quarter ID').asc()).toPandas())

In [0]:
# union_df.groupBy('Fiscal Period ID').agg({'CA Service Bookings Net': 'sum'}).orderBy(F.col('Fiscal Period ID').asc()).show()
display(union_df.groupBy('Fiscal Period ID').agg({'CA Service Bookings Net': 'sum'}).orderBy(F.col('Fiscal Period ID').asc()).toPandas())

Fiscal Period ID,sum(CA Service Bookings Net)
201811,1.673123449334962E8
201812,5.54619008100895E8
201903,2.4110930672846067E8
201906,3.2265350458538646E8


In [0]:
# Add BE and FileName col
from pyspark.sql.functions import lit
from  pyspark.sql.functions import input_file_name

union_df = union_df.withColumn("FileName", concat(input_file_name(),lit("NewNew"))).withColumn("BE", lit("Enterprise Switching"))
# union_df.select("FileName").collect()[1]

In [0]:
union_df.coalesce(1).write.format("com.databricks.spark.csv").mode("overwrite").option("header","true").option("sep","|").save("dbfs:/FileStore/tables/Carmel/TSS_Collaboration.csv")

In [0]:
union_df2 = spark.read.format("csv") \
                             .option("header", "true") \
                             .option("delimiter", "|") \
                             .option("inferSchema","true") \
                             .option("quote", "\"") \
                             .option("escape", "\"") \
                             .load('dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_EnterpriseSwitching4.csv/part-00000-tid-23867787626438659-c8bc6ca3-bded-4ee0-9542-c7f18a9c7e94-205348-1-c000.csv') \
                             .filter( F.col("Fiscal Period ID") == 201808) 
#                              .filter( (F.col("Fiscal Year") == 2018) | (F.col("Fiscal Year") == 2021) )

In [0]:
display(union_df2.groupBy('Fiscal Period ID').agg({'CA Service Bookings Net': 'sum'}).orderBy(F.col('Fiscal Period ID').asc()).toPandas())

Fiscal Period ID,sum(CA Service Bookings Net)
201808,1.3420121653706576E8


In [0]:
# Add BE and FileName col
from pyspark.sql.functions import lit
from  pyspark.sql.functions import input_file_name

union_df2 = union_df2.withColumn("FileName", concat(input_file_name(),lit("New"))).withColumn("BE", lit("Enterprise Switching"))
union_df2.show()

+-----------+--------------+------------------+-----------------+----------------+---------------------------+----------------------------------+-------------------------+----------------------------+--------------------------+-------------------------+-----------------------+---------------------------+-------------------------+-----------------------+-----------------+------------------+--------------------+-------------------------+-----------------------+---------------------------------+---------------------+-------------------+----------------------+------------+-----------+-------------+-------------+--------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+--------------------+
Fiscal Year| Product ID| PH-PID|Fiscal Quarter ID|Fiscal Period ID|Finance BU/Service Category|Finance Product Family/Allctd Svcs|Bookings Transaction Type|Bill To Global Ultimate Name|Bill To Global Ultimate ID|ERP Bill To Customer Name|ERP Bill To Customer Id|ERP Bill To Customer Number|ERP Sold To Customer Name|ERP Sold To Customer Id|Ship To Site City|Ship To Site State|Ship To Site Country|ERP Ship To Customer Name|ERP Ship To Customer Id|End Customer Global Ultimate Name|ERP End Customer Name|ERP End Customer Id|ERP Sales Order Number|Web Order ID|ERP Deal ID| Country|Sales Level 1| Sales Level 2| Sales Level 3| Sales Level 4|CA Service Bookings Net| TSS Methodology| FileName| BE|
+-----------+--------------+------------------+-----------------+----------------+---------------------------+----------------------------------+-------------------------+----------------------------+--------------------------+-------------------------+-----------------------+---------------------------+-------------------------+-----------------------+-----------------+------------------+--------------------+-------------------------+-----------------------+---------------------------------+---------------------+-------------------+----------------------+------------+-----------+-------------+-------------+--------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+--------------------+
 2018| CON-NCET-1| WS-C3750X-24T-L| 2018Q3| 201808| TECHNICAL SUPPORT...| AS CORE| AR| GENERAL DATATECH LP| 2654297| GENERAL DATATECH LP| 750723| 750383| GENERAL DATATECH LP| 750723| BARTLESVILLE| OK| UNITED STATES| PHILLIPS 66| 133227747| PHILLIPS 66| PHILLIPS 66| 133227747| 105720445| UNKNOWN| 17867304|UNITED STATES| Americas|GLOBAL ENTERPRISE...| GES SOUTH| RED RIVER OPERATION| -0.019619138|Prior Period Orde...|dbfs:/mnt/files/i...|Enterprise Switching|
 2018| CON-SNT-SMS-1|C1-WSC3850-12X48UL| 2018Q3| 201808| TECHNICAL SUPPORT...| SMART NET TOTAL CARE| POS| PROUSYS INC.| 11198420| PROUSYS INC| 5794386| UNKNOWN| PROUSYS INC| 5794386| BAKERSFIELD| CA| UNITED STATES| PROUSYS INC| 5794386| MAINE COMMUNITY B...| MAINE COMMUNITY B...| 13443970| -9999| UNKNOWN| null|UNITED STATES| Americas| US COMMERCIAL|SBU_US COMMERCIAL...|SBU_USC EAST SMAL...| 1.703975|Country & SKU Lev...|dbfs:/mnt/files/i...|Enterprise Switching|
 2018| CON-SNT-SMS-1| WS-C4507RE+96V+| 2018Q3| 201808| TECHNICAL SUPPORT...| SMART NET TOTAL CARE| POS| REVEL SOLUTIONS LLC| 181742558| REVEL SOLUTIONS LLC| 9002072| UNKNOWN| REVEL SOLUTIONS LLC| 9002072| HOUSTON| TX| UNITED STATES| REVEL SOLUTIONS LLC| 9002072| PANTHER CREEK MED...| VILLAGE MEDICAL C...| 8059596| -9999| UNKNOWN| null|UNITED STATES| Americas| US COMMERCIAL|SBU_US COMMERCIAL...|SBU_USC SOUTH SMA...| 2.104331|Country & SKU Lev...|dbfs:/mnt/files/i...|Enterprise Switching|
 2018| CON-SNT-SMS-1| WS-C3850-48U-S| 2018Q3| 201808| TECHNICAL SUPPORT...| SMART NET TOTAL CARE| POS| PERSONAL COMPUTER...| 160899419| PERSONAL COMPUTER...| 2401190| UNKNOWN| PERSONAL COMPUTER...| 2401190| BUFFALO| NY| UNITED STATES| PERSONAL COMPUTER...| 2401190| JW DANSWORTH| J W DANFORTH| 13603167| -9999| UNKNOWN| null|UNITED STATES| Americas| US

In [0]:
union_df3 = unionAll(union_df, union_df2)

In [0]:
display(union_df3.groupBy('Fiscal Quarter ID').agg({'CA Service Bookings Net': 'sum'}).orderBy(F.col('Fiscal Quarter ID').asc()).toPandas())

Fiscal Quarter ID,sum(CA Service Bookings Net)
2016Q1,2.4267208312849972E8
2016Q2,3.223203690986485E8
2016Q3,3.3286474550748533E8
2016Q4,5.323277521808485E8
2017Q1,1.9914259332420686E8
2017Q2,4.585708117257401E8
2017Q3,2.4202035632683125E8
2017Q4,4.2826060809983844E8
2018Q1,2.1288992536497587E8
2018Q2,3.462841027653584E8


In [0]:
union_df.columns

Out[2]: ['Fiscal Year',
 'Product ID',
 'PH-PID',
 'Fiscal Quarter ID',
 'Fiscal Period ID',
 'Finance BU/Service Category',
 'Finance Product Family/Allctd Svcs',
 'Bookings Transaction Type',
 'Bill To Global Ultimate Name',
 'Bill To Global Ultimate ID',
 'ERP Bill To Customer Name',
 'ERP Bill To Customer Id',
 'ERP Bill To Customer Number',
 'ERP Sold To Customer Name',
 'ERP Sold To Customer Id',
 'Ship To Site City',
 'Ship To Site State',
 'Ship To Site Country',
 'ERP Ship To Customer Name',
 'ERP Ship To Customer Id',
 'End Customer Global Ultimate Name',
 'ERP End Customer Name',
 'ERP End Customer Id',
 'ERP Sales Order Number',
 'Web Order ID',
 'ERP Deal ID',
 'Country',
 'Sales Level 1',
 'Sales Level 2',
 'Sales Level 3',
 'Sales Level 4',
 'CA Service Bookings Net',
 'TSS Methodology',
 'BE',
 'FileName']

In [0]:
# Reorder columns to match target table
union_df = union_df.select('Fiscal Year',
 'Product ID',
 'PH-PID',
 'Fiscal Quarter ID',
 'Fiscal Period ID',
 'Finance BU/Service Category',
 'Finance Product Family/Allctd Svcs',
 'Bookings Transaction Type',
 'Bill To Global Ultimate Name',
 'Bill To Global Ultimate ID',
 'ERP Bill To Customer Name',
 'ERP Bill To Customer Id',
 'ERP Bill To Customer Number',
 'ERP Sold To Customer Name',
 'ERP Sold To Customer Id',
 'Ship To Site City',
 'Ship To Site State',
 'Ship To Site Country',
 'ERP Ship To Customer Name',
 'ERP Ship To Customer Id',
 'End Customer Global Ultimate Name',
 'ERP End Customer Name',
 'ERP End Customer Id',
 'ERP Sales Order Number',
 'Web Order ID',
 'ERP Deal ID',
 'Country',
 'Sales Level 1',
 'Sales Level 2',
 'Sales Level 3',
 'Sales Level 4',
 'CA Service Bookings Net',
 'TSS Methodology',
 'FileName',
 'BE'
 )

In [0]:
union_df.coalesce(1).write.format("com.databricks.spark.csv").mode("overwrite").option("header","true").option("sep","|").save("dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_EnterpriseSwitching7.csv")

In [0]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
# import com.microsoft.kusto.spark.datasource.KustoOptions
# import com.microsoft.kusto.spark.sql.extension.SparkExtension._
# import org.apache.spark.SparkConf
# import org.apache.spark.sql._
import pandas as pd

In [0]:
pip install azure-functions

Python interpreter will be restarted.
Collecting azure-functions
 Downloading azure_functions-1.7.2-py3-none-any.whl (137 kB)
Installing collected packages: azure-functions
Successfully installed azure-functions-1.7.2
WARNING: You are using pip version 20.2.4; however, version 21.2.4 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-9c87fcb6-a49d-4429-a339-963d35dc36b8/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
pip install azure-kusto-data azure-kusto-ingest

Python interpreter will be restarted.
Collecting azure-kusto-data
 Downloading azure_kusto_data-2.3.1-py2.py3-none-any.whl (31 kB)
Collecting azure-kusto-ingest
 Downloading azure_kusto_ingest-2.3.1-py2.py3-none-any.whl (17 kB)
Requirement already satisfied: python-dateutil>=2.8.0 in /databricks/python3/lib/python3.8/site-packages (from azure-kusto-data) (2.8.1)
Collecting msal<2,>=1.9.0
 Downloading msal-1.14.0-py2.py3-none-any.whl (75 kB)
Collecting azure-identity<1.7.0,>=1.5.0
 Downloading azure_identity-1.6.1-py2.py3-none-any.whl (109 kB)
Requirement already satisfied: requests>=2.13.0 in /databricks/python3/lib/python3.8/site-packages (from azure-kusto-data) (2.24.0)
Collecting azure-storage-blob<13,>=12
 Downloading azure_storage_blob-12.9.0-py2.py3-none-any.whl (356 kB)
Collecting azure-storage-queue<13,>=12
 Downloading azure_storage_queue-12.1.6-py2.py3-none-any.whl (137 kB)
Requirement already satisfied: six>=1.5 in /databricks/python3/lib/python3.8/site-packages (from python-dateutil>=2.8.0->azure-kusto-data) (1.15.0)
Requirement already satisfied: PyJWT[crypto]<3,>=1.0.0 in /databricks/python3/lib/python3.8/site-packages (from msal<2,>=1.9.0->azure-kusto-data) (2.1.0)
Requirement already satisfied: cryptography<4,>=0.6 in /databricks/python3/lib/python3.8/site-packages (from msal<2,>=1.9.0->azure-kusto-data) (3.1.1)
Collecting azure-core<2.0.0,>=1.0.0
 Downloading azure_core-1.18.0-py2.py3-none-any.whl (166 kB)
Collecting msal-extensions~=0.3.0
 Downloading msal_extensions-0.3.0-py2.py3-none-any.whl (16 kB)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.13.0->azure-kusto-data) (1.25.11)
Requirement already satisfied: chardet<4,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.13.0->azure-kusto-data) (3.0.4)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.13.0->azure-kusto-data) (2020.12.5)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.13.0->azure-kusto-data) (2.10)
Collecting msrest>=0.6.21
 Downloading msrest-0.6.21-py2.py3-none-any.whl (85 kB)
Requirement already satisfied: cffi!=1.11.3,>=1.8 in /databricks/python3/lib/python3.8/site-packages (from cryptography<4,>=0.6->msal<2,>=1.9.0->azure-kusto-data) (1.14.3)
Collecting portalocker~=1.0; platform_system != "Windows"
 Downloading portalocker-1.7.1-py2.py3-none-any.whl (10 kB)
Collecting isodate>=0.6.0
 Downloading isodate-0.6.0-py2.py3-none-any.whl (45 kB)
Collecting requests-oauthlib>=0.5.0
 Downloading requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
Requirement already satisfied: pycparser in /databricks/python3/lib/python3.8/site-packages (from cffi!=1.11.3,>=1.8->cryptography<4,>=0.6->msal<2,>=1.9.0->azure-kusto-data) (2.20)
Collecting oauthlib>=3.0.0
 Downloading oauthlib-3.1.1-py2.py3-none-any.whl (146 kB)
Installing collected packages: msal, azure-core, portalocker, msal-extensions, azure-identity, azure-kusto-data, isodate, oauthlib, requests-oauthlib, msrest, azure-storage-blob, azure-storage-queue, azure-kusto-ingest
Successfully installed azure-core-1.18.0 azure-identity-1.6.1 azure-kusto-data-2.3.1 azure-kusto-ingest-2.3.1 azure-storage-blob-12.9.0 azure-storage-queue-12.1.6 isodate-0.6.0 msal-1.14.0 msal-extensions-0.3.0 msrest-0.6.21 oauthlib-3.1.1 portalocker-1.7.1 requests-oauthlib-1.3.0
WARNING: You are using pip version 20.2.4; however, version 21.2.4 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-9c87fcb6-a49d-4429-a339-963d35dc36b8/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
# Get credentials
adxTenantId = dbutils.secrets.get(scope = "ce5", key = "adxTenantId")
adxClientId = dbutils.secrets.get(scope = "ce5", key = "adxClientId")
adxClientSecret = dbutils.secrets.get(scope = "ce5", key = "adxClientSecret")
adxClusterName = dbutils.secrets.get(scope = "ce5", key = "adxClusterName")
adxDatabaseName = dbutils.secrets.get(scope = "ce5", key = "adxDatabaseName")

In [0]:
# Establish connection
from pyspark.sql import SparkSession

pyKusto = SparkSession.builder.appName("kustoPySpark").getOrCreate()

kustoOptions = {\
  "kustoCluster": adxClusterName,\
  "kustoDatabase": adxDatabaseName,\
  "kustoAadAppId": adxClientId,\
  "kustoAadAppSecret": adxClientSecret,\
  "kustoAadAuthorityID": adxTenantId\
}

In [0]:
# read data from kusto
query = "ServiceData | take 10"

kustoDf  = spark.read. \
  format("com.microsoft.kusto.spark.datasource"). \
  option("kustoCluster", kustoOptions["kustoCluster"]). \
  option("kustoDatabase", kustoOptions["kustoDatabase"]). \
  option("kustoQuery", query). \
  option("kustoAadAppId", kustoOptions["kustoAadAppId"]). \
  option("kustoAadAppSecret", kustoOptions["kustoAadAppSecret"]). \
  option("kustoAadAuthorityID", kustoOptions["kustoAadAuthorityID"]). \
  load()

display(kustoDf)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1554056150930453> in <module> 
 2 query = "ServiceData | take 10" 
 3 
 ----> 4 kustoDf = spark . read . \ 
 5 format ( "com.microsoft.kusto.spark.datasource" ) . \ 
 6 option ( "kustoCluster" , kustoOptions [ "kustoCluster" ] ) . \ 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 208 return self . _df ( self . _jreader . load ( self . _spark . _sc . _jvm . PythonUtils . toSeq ( path ) ) ) 
 209 else : 
 --> 210 return self . _df ( self . _jreader . load ( ) ) 
 211 
 212 def json(self, path, schema=None, primitivesAsString=None, prefersDecimal=None,

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o318.load.
: java.lang.NoClassDefFoundError: com/microsoft/azure/kusto/data/exceptions/DataServiceException
	at com.microsoft.kusto.spark.datasource.DefaultSource.createRelation(DefaultSource.scala:95)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:390)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:432)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$3(DataFrameReader.scala:399)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:399)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:272)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.ClassNotFoundException: com.microsoft.azure.kusto.data.exceptions.DataServiceException
	at java.net.URLClassLoader.findClass(URLClassLoader.java:382)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:419)
	at com.databricks.backend.daemon.driver.ClassLoaders$LibraryClassLoader.loadClass(ClassLoaders.scala:151)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:352)
	... 18 more

In [0]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder


def getADXConnection():
  # Get connection parameters
  cluster_name = dbutils.secrets.get(scope = "ce5", key = "adxClusterName")
  client_id = dbutils.secrets.get(scope = "ce5", key = "adxClientId")
  client_secret = dbutils.secrets.get(scope = "ce5", key = "adxClientSecret")
  authority_id = dbutils.secrets.get(scope = "ce5", key = "adxTenantId")
  # Build connection
  kcsb = KustoConnectionStringBuilder.with_aad_application_key_authentication("https://" + cluster_name + ".kusto.windows.net",
        client_id,
        client_secret,
        authority_id)

  # create a client from the connection details
  return KustoClient(kcsb)

In [0]:
from azure.kusto.data.helpers import dataframe_from_result_table

client = getADXConnection()
cluster = "https://USEPTASSSPADE01.eastus.kusto.windows.net"
db = "confidential-projectcarmel-2252bd6d-e9f5-4ade-8373-abb4f8d2f"
query = "ServiceData | take 1000"
response = client.execute(db, query)
# execute the query
query = "set notruncation; "+query
response = client.execute(db, query)
# convert to pandas dataframe
dfk = dataframe_from_result_table(response.primary_results[0])
# df1.insert(6, 'TimeStamp', dateStr)

#drop all null columns
dfk = dfk.dropna(axis='columns', how='all') 

print(dfk.columns)
dfk.head()

Index(['FiscalYear', 'ProductID', 'PHPID', 'FiscalQuarterID', 'FiscalPeriodID',
 'FinanceBUServiceCategory', 'FinanceProductFamilyAllctdSvcs',
 'BookingsTransactionType', 'BillToGlobalUltimateName',
 'BillToGlobalUltimateID', 'ERPBillToCustomerName',
 'ERPBillToCustomerId', 'ERPBillToCustomerNumber',
 'ERPSoldToCustomerName', 'ERPSoldToCustomerId', 'ShipToSiteCountry',
 'ERPShipToCustomerName', 'ERPShipToCustomerId',
 'EndCustomerGlobalUltimateName', 'ERPEndCustomerName',
 'ERPEndCustomerId', 'ERPSalesOrderNumber', 'WebOrderID', 'ERPDealID',
 'Country', 'SalesLevel1', 'SalesLevel2', 'SalesLevel3', 'SalesLevel4',
 'CAServiceBookingsNet', 'TSSMethodology', 'FileName', 'ShipToSiteCity',
 'ShipToSiteState', 'BookingsRevenue_EY', 'FinanceSubGroupContractType',
 'TransactionType', 'AdjustmentTypeCode', 'L1SalesFinanceHierarchyCode',
 'L2SalesFinanceHierarchyCode', 'L3SalesFinanceHierarchyCode',
 'ERPShipToCustomerNumber', 'ERPEndCustomerNumber', 'GLAccountNumber',
 'iris_id', 'iris_metadata'],
 dtype='object')
Out[7]:

,FiscalYear,ProductID,PHPID,FiscalQuarterID,FiscalPeriodID,FinanceBUServiceCategory,FinanceProductFamilyAllctdSvcs,BookingsTransactionType,BillToGlobalUltimateName,BillToGlobalUltimateID,ERPBillToCustomerName,ERPBillToCustomerId,ERPBillToCustomerNumber,ERPSoldToCustomerName,ERPSoldToCustomerId,ShipToSiteCountry,ERPShipToCustomerName,ERPShipToCustomerId,EndCustomerGlobalUltimateName,ERPEndCustomerName,ERPEndCustomerId,ERPSalesOrderNumber,WebOrderID,ERPDealID,Country,SalesLevel1,SalesLevel2,SalesLevel3,SalesLevel4,CAServiceBookingsNet,TSSMethodology,FileName,ShipToSiteCity,ShipToSiteState,BookingsRevenue_EY,FinanceSubGroupContractType,TransactionType,AdjustmentTypeCode,L1SalesFinanceHierarchyCode,L2SalesFinanceHierarchyCode,L3SalesFinanceHierarchyCode,ERPShipToCustomerNumber,ERPEndCustomerNumber,GLAccountNumber,iris_id,iris_metadata
0,2016,CON-PSUP-SMS-1,TR-SP-5108-DC3-TA,2016Q4,201612,TECHNICAL SUPPORT SERVICES,PARTNER SUPPORT SERVICE,AR,COMTRADE HOLDING NV,192226362,COMTRADE DISTRIBUTION,1000026016,1000026017,UNKNOWN,-999,SERBIA,COMTRADE DISTRIBUTION,1000026016,UNKNOWN,UNKNOWN,-999,-7777,UNKNOWN,,UNKNOWN,WW Distribution,EMEA_Distribution_,EMEA Central Disti,EMEA Central RGN_Disti,-0.282873,Country Allocation,TSS_BOOKING_CARMEL_2021Q3_SOonly_Computing sys...,BEOGRAD,,Bookings,,,,,,,,,,3dea0070-3cb6-4468-81de-9ad982f22c76,"{'uploadedBy': {'name': 'Fortuna Zhang', 'id':..."
1,2016,CON-SNT-1,UCS-EZ7-B200-P,2016Q4,201612,TECHNICAL SUPPORT SERVICES,SMART NET TOTAL CARE,AR,NIPPON TELEGRAPH AND TELEPHONE CORPORATION,147183,DIMENSION DATA CHINA/HONG KONG LIMITED,120037524,55537182,DIMENSION DATA CHINA/HONG KONG LIMITED,120037524,HONG KONG,Dabserve Pty Ltd,131599036,DABSERVE PTY LTD,Dabserve Pty Ltd,131599036,102486061,UNKNOWN,,HONG KONG,APJC,GREATER_CHINA,HONG_KONG,HK_COMMERCIAL,-10.159222,Prior Period Order (O2Q) Mapping,TSS_BOOKING_CARMEL_2021Q3_SOonly_Computing sys...,HONG KONG,HK,Bookings,,,,,,,,,,95b06a54-7a3a-4968-aba3-92049b48b0af,"{'uploadedBy': {'name': 'Fortuna Zhang', 'id':..."
2,2016,CON-SNTP-SMS-1,R210-2121605W=,2016Q4,201612,TECHNICAL SUPPORT SERVICES,SMART NET TOTAL CARE,ERP,"TIANJIN MARINE SHIPPING CO., LTD.",231122301,INGRAM MICRO,10239,10140,INGRAM MICRO,10239,UNITED STATES,INGRAM MICRO,10239,DISTI STOCK,INGRAM MICRO,10239,102751478,UNKNOWN,,UNITED STATES,WW Distribution,Americas Distribution,US Distribution L3,US Distribution L4,513.294388,Order to Quote Mapping,TSS_BOOKING_CARMEL_2021Q3_SOonly_Computing sys...,CAROL STREAM,IL,Bookings,,,,,,,,,,fc4aa187-95d7-4f39-957d-8d402e33be0b,"{'uploadedBy': {'name': 'Fortuna Zhang', 'id':..."
3,2016,CON-PSJ3-1,R250-2480805W,2016Q4,201612,TECHNICAL SUPPORT SERVICES,PARTNER SUPPORT SERVICE,ERP,RING LINE CORPORATION,2160250,RING LINE CORPORATION,120029657,55528914,RING LINE CORPORATION,120029657,TAIWAN,DAWNING LEADING TECHNOLOGY INC,132473484,KING YUAN ELECTRONICS COMPANY LTD,DAWNING LEADING TECHNOLOGY INC,132473484,102791647,UNKNOWN,,TAIWAN,APJC,GREATER_CHINA,TAIWAN,GLL_FOXCONN,440.520000,Order to Quote Mapping,TSS_BOOKING_CARMEL_2021Q3_SOonly_Computing sys...,ZHUNAN TOWNSHIP,MIAOLI COUNTY,Bookings,,,,,,,,,,b7a97777-be32-47c8-a445-419f43b3f9ef,"{'uploadedBy': {'name': 'Fortuna Zhang', 'id':..."
4,2016,CON-PSJ1-1,UCSC-C22-M3S,2016Q4,201612,TECHNICAL SUPPORT SERVICES,PARTNER SUPPORT SERVICE,ERP,ADVANCE INFORMATION TECHNOLOGY COLTD,37877104,ADVANCED INFORMATION TECHNOLOGY PUBLIC COMPANY...,120043818,55543759,ADVANCED INFORMATION TECHNOLOGY PUBLIC COMPANY...,120043818,THAILAND,THE MINISTRY OF SCIENCE TECHNOLOGY AND ENVIRON...,132010834,THE KINGDOM OF THAILAND,THE MINISTRY OF SCIENCE TECHNOLOGY AND ENVIRON...,132010834,102707402,UNKNOWN,,THAILAND,APJC,ASEAN_AREA,ASEAN_TH,ASEAN_PS_THA,123.160000,Order to Quote Mapping,TSS_BOOKING_CARMEL_2021Q3_SOonly_Computing sys...,BANGKOK,BANGKOK,Bookings,,,,,,,,,,9ac90040-4405-45ba-b293-c41c421cf94b,"{'uploadedBy': {'name': 'Fortuna Zhang', 'id':..."


In [0]:
# Write the df to Kusto
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .appName("kustoPySpark") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas

sdfk = spark.createDataFrame(dfk)

sdfk.write.format("com.microsoft.kusto.spark.datasource"). \
    option("kustoCluster", cluster). \
    option("kustoDatabase", db). \
    option("kustoAadAppId", adxClientId). \
    option("kustoAadAppSecret", adxClientSecret). \
    option("kustoAadAuthorityID", adxTenantId). \
    option("kustoTable", "DBS_Test"). \
    mode("Append"). \
    save()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:300: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Unable to convert the field iris_metadata. If this column is not necessary, you may consider dropping it or converting to primitive type before the conversion.
Direct cause: Nested StructType not supported in conversion from Arrow: struct<filename: string, uploadId: string, uploadedBy: struct<id: string, name: string>, uploadedOn: string>
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1554056150930443> in <module> 
 9 sdfk = spark . createDataFrame ( dfk ) 
 10 
 ---> 11 sdfk . write . format ( "com.microsoft.kusto.spark.datasource" ) . \ 
 12 option ( "kustoCluster" , cluster ) . \ 
 13 option ( "kustoDatabase" , db ) . \ 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 1132 self . format ( format ) 
 1133 if path is None : 
 -> 1134 self . _jwrite . save ( ) 
 1135 else : 
 1136 self . _jwrite . save ( path ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o449.save.
: java.lang.NoClassDefFoundError: com/microsoft/azure/kusto/data/exceptions/DataServiceException
	at com.microsoft.kusto.spark.datasource.DefaultSource.createRelation(DefaultSource.scala:36)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:48)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:73)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.doExecute(commands.scala:96)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:196)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:240)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:165)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:236)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:192)
	at org.apache.spark.sql.execution.QueryExecution.toRdd$lzycompute(QueryExecution.scala:167)
	at org.apache.spark.sql.execution.QueryExecution.toRdd(QueryExecution.scala:166)
	at org.apache.spark.sql.DataFrameWriter.$anonfun$runCommand$1(DataFrameWriter.scala:1079)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$5(SQLExecution.scala:126)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:267)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$1(SQLExecution.scala:104)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:852)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:77)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:217)
	at org.apache.spark.sql.DataFrameWriter.runCommand(DataFrameWriter.scala:1079)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:468)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:438)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:311)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAc

In [0]:
%python
dir_path = '/mnt/files/ir/files/Bookings/Consolidated/'
src_files = []
def walk_dir(dir_path):
  dir_files = dbutils.fs.ls(dir_path)
  for file in dir_files:
    if file.isDir():
      walk_dir(file.path)
    else: src_files.append(file.path)

In [0]:
walk_dir('/mnt/files/ir/files/Bookings/Consolidated/')

src_files

Out[10]: ['dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_Collaboration.csv/_SUCCESS',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_Collaboration.csv/_committed_7293162403850464639',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_Collaboration.csv/_committed_999819933976769861',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_Collaboration.csv/_committed_vacuum1296148470380756029',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_Collaboration.csv/_started_7293162403850464639',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_Collaboration.csv/part-00000-tid-7293162403850464639-300af59c-c71a-4db6-bc17-cb19a16884f3-265-1-c000.csv',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_DataCenterNetwork.csv/_SUCCESS',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_DataCenterNetwork.csv/_committed_2416728681388419389',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_DataCenterNetwork.csv/_committed_6111218569358834044',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_DataCenterNetwork.csv/_committed_vacuum3628609350360950360',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_DataCenterNetwork.csv/_started_2416728681388419389',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_DataCenterNetwork.csv/_started_6671859014377124157',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_DataCenterNetwork.csv/part-00000-tid-2416728681388419389-41aefb60-c4c8-4c90-ab9a-4424eaa438f4-2099067-1-c000.csv',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_DataCenterNetwork.csv/part-00000-tid-6671859014377124157-cc5fe123-0f9c-4c8f-b0a1-0bc707e2754f-2092915-1-c000.csv',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_EnterpriseRouting.csv/_SUCCESS',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_EnterpriseRouting.csv/_committed_3900534477168572665',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_EnterpriseRouting.csv/_started_3900534477168572665',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_EnterpriseRouting.csv/part-00000-tid-3900534477168572665-69c696be-0a73-4a20-9a37-536be355adfe-2135525-1-c000.csv',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_Security.csv/_SUCCESS',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_Security.csv/_committed_930590702649658854',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_Security.csv/_started_930590702649658854',
 'dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_Security.csv/part-00000-tid-930590702649658854-4e1c6e0f-52ee-47c0-a556-68525794987e-737432-1-c000.csv']

In [0]:
check_df = spark.read.format("csv") \
                             .option("header", "true") \
                             .option("delimiter", "|") \
                             .option("inferSchema","true") \
                             .option("ESCAPE quote", '"') \
                             .load('dbfs:/mnt/files/ir/files/Bookings/Consolidated/TSS_Collaboration.csv/part-00000-tid-7293162403850464639-300af59c-c71a-4db6-bc17-cb19a16884f3-265-1-c000.csv')

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window
from pyspark.sql import functions as F

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

display(check_df.groupBy('Fiscal Quarter ID').agg({'CA Service Bookings Net': 'sum'}).orderBy(F.col('Fiscal Quarter ID').asc()).toPandas())

Fiscal Quarter ID,sum(CA Service Bookings Net)
2016Q1,2.0028185250427595E8
2016Q2,3.4992963968306464E8
2016Q3,3.568502076510465E8
2016Q4,5.2324022125461257E8
2017Q1,2.6923044822641456E8
2017Q2,3.912933502237598E8
2017Q3,3.413855180546724E8
2017Q4,5.65476145464163E8
2018Q1,3.1218702186394835E8
2018Q2,3.599555146121653E8


In [0]:
display(check_df.filter(col("Fiscal Year") == "2016").groupBy('FileName').agg({'CA Service Bookings Net': 'sum'}).orderBy(F.col('FileName').asc()).toPandas())

FileName,sum(CA Service Bookings Net)
dbfs:/mnt/files/ir/files/Bookings/Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201601.csv,1.960069746570016E7
dbfs:/mnt/files/ir/files/Bookings/Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201602.csv,5.955641845200573E7
dbfs:/mnt/files/ir/files/Bookings/Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201603.csv,1.2112473658658446E8
dbfs:/mnt/files/ir/files/Bookings/Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201604.csv,6.779736771154253E7
dbfs:/mnt/files/ir/files/Bookings/Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201605.csv,9.741905441444819E7
dbfs:/mnt/files/ir/files/Bookings/Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201606.csv,1.8471321755705774E8
dbfs:/mnt/files/ir/files/Bookings/Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201607.csv,8.348643394743669E7
dbfs:/mnt/files/ir/files/Bookings/Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201608.csv,7.59861258467875E7
dbfs:/mnt/files/ir/files/Bookings/Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201609.csv,1.9737764785682964E8
dbfs:/mnt/files/ir/files/Bookings/Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201610.csv,5.246289272148833E7


In [0]:
subcheck_df = spark.read.format("csv") \
                             .option("header", "true") \
                             .option("delimiter", ",") \
                             .option("inferSchema","true") \
                             .option("ESCAPE quote", '"') \
                             .load('dbfs:/mnt/files/ir/files/Bookings/Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201602.csv')

In [0]:
subcheck_df.show()

+-----------+-----------------+--------------------+-----------------+----------------+---------------------------+----------------------------------+-------------------------+----------------------------+--------------------------+-------------------------+-----------------------+---------------------------+-------------------------+-----------------------+-----------------+------------------+--------------------+-------------------------+-----------------------+---------------------------------+---------------------+-------------------+----------------------+------------+-----------+------------------+---------------+--------------------+--------------------+--------------------+-----------------------+--------------------+
Fiscal Year| Product ID| PH-PID|Fiscal Quarter ID|Fiscal Period ID|Finance BU/Service Category|Finance Product Family/Allctd Svcs|Bookings Transaction Type|Bill To Global Ultimate Name|Bill To Global Ultimate ID|ERP Bill To Customer Name|ERP Bill To Customer Id|ERP Bill To Customer Number|ERP Sold To Customer Name|ERP Sold To Customer Id|Ship To Site City|Ship To Site State|Ship To Site Country|ERP Ship To Customer Name|ERP Ship To Customer Id|End Customer Global Ultimate Name|ERP End Customer Name|ERP End Customer Id|ERP Sales Order Number|Web Order ID|ERP Deal ID| Country| Sales Level 1| Sales Level 2| Sales Level 3| Sales Level 4|CA Service Bookings Net| TSS Methodology|
+-----------+-----------------+--------------------+-----------------+----------------+---------------------------+----------------------------------+-------------------------+----------------------------+--------------------------+-------------------------+-----------------------+---------------------------+-------------------------+-----------------------+-----------------+------------------+--------------------+-------------------------+-----------------------+---------------------------------+---------------------+-------------------+----------------------+------------+-----------+------------------+---------------+--------------------+--------------------+--------------------+-----------------------+--------------------+
 2016| CON-ECDN-1| CTS-PHD-1080P-KIT=| 2016Q1| 201601| TECHNICAL SUPPORT...| CISCO OPERATE SER...| AR | CDW HOLDINGS LLC| 171165710| CDW LOGISTICS LLC| 400488510| 400434104| CDW LOGISTICS LLC| 400488510| FORT WORTH| TX| UNITED STATES| MEDTRONIC| 401604803| MEDTRONIC INC| MEDTRONIC| 401604803| 100867377| UNKNOWN| 14390813| UNITED KINGDOM| EMEAR-REGION| EMEAR-UKI| COMMERCIAL_EAW| SMALL_EAW_S| -0.001033|Prior Period Orde...|
 2016| CON-PSRN-1| CTS-SX20-4XK9-PRM| 2016Q1| 201601| TECHNICAL SUPPORT...| PARTNER SUPPORT S...| ERP | NIPPON TELEGRAPH ...| 147183| DIMENSION DATA CH...| 120037524| 55537182| DIMENSION DATA CH...| 120037524| HONG KONG| HK| HONG KONG| LVMH ASIA PACIFIC...| 650600| LVMH MOET HENNESS...| LVMH ASIA PACIFIC...| 650600| 101051837| UNKNOWN| null| HONG KONG| APJC| GREATER_CHINA| HONG_KONG| HK_COMMERCIAL| 258.09|Order to Quote Ma...|
 2016|CON-ECDN-SX2PHD4X| CTS-SX20-PHD4X-K9| 2016Q1| 201601| TECHNICAL SUPPORT...| CISCO OPERATE SER...| AR | CDW HOLDINGS LLC| 171165710| CDW LOGISTICS LLC| 400488510| 400434104| CDW LOGISTICS LLC| 400488510| HOUSTON| TX| UNITED STATES| GDF SUEZ ENERGY N...| 403649240| ENGIE| GDF SUEZ ENERGY N...| 403649240| 100764189| UNKNOWN| 15766235| UNITED STATES| EMEAR-REGION| EMEAR-SOUTH| COUNTRY_FRANCE| CGEM_FRA| -13.89| SMT Mapping|
 2016| CON-SNT-SMS-1| CP-7841-K9=| 2016Q1| 201601| TECHNICAL SUPPORT...| SMART NET TOTAL CARE| POS | EZE CASTLE INTEGR...| 4915690| EZE CASTLE INTEGR...| 2401309| UNKNOWN| EZE CASTLE INTEGR...| 2401309| BOSTON| MA| UNITED STATES| EZE CASTLE INTEGR...| 2401309| PAGODA ASSET MANA...| PAGODA ASSET MANA...| 10259665| -9999| UNKNOWN| null| UNITED STATES| Americas| US COMMERCIAL|SBU_US COMMERCIAL...|SBU_USC EAST SMAL...| 1.693503|Country & SKU Lev...|
 2016| CON-SNT-1| CP-7925G-W-K9=| 2016Q1| 201601| TECHNICAL SUPPORT...| SMART NET TOTAL CARE| ERP | TELEPHONE

In [0]:
display(subcheck_df.groupBy("Fiscal Period ID").agg({"CA Service Bookings Net": 'sum'}).orderBy(F.col("Fiscal Period ID").asc()).toPandas())

Fiscal Period ID,sum(CA Service Bookings Net)
201602,5.9556418452006176E7


In [0]:
display(subcheck_df.groupBy("TSS Methodology").agg({"CA Service Bookings Net": 'sum'}).orderBy(F.col("TSS Methodology").asc()).toPandas())

TSS Methodology,sum(CA Service Bookings Net)
null,null
CMS SKU to BE Mapping,968834.7528839998
Country & SKU Level Allocation,-49221.429904000026
Country Allocation,-2431035.3811313156
Enriched POS O2Q Mapping,94379.86846099995
GSP allocation,432.105291
GSP to BE Mapping,-1392791.0615861164
Order to Quote Mapping,1.931958708541935E7
Prior Period Order (O2Q) Mapping,-1040643.1718855754
SKU Level allocation,66988.76672100002


In [0]:
subcheck_df1 = subcheck_df.withColumn("RN", row_number().over(Window.orderBy(monotonically_increasing_id()))-1)

print(subcheck_df1.select("CA Service Bookings Net", F.col("CA Service Bookings Net").cast('float').isNotNull().alias("IfNumber")).filter("IfNumber == false").count())
subcheck_df1.select("RN", "CA Service Bookings Net", F.col("CA Service Bookings Net").cast('float').isNotNull().alias("IfNumber")).filter("IfNumber == false").show()

1
+------+-----------------------+--------+
 RN|CA Service Bookings Net|IfNumber|
+------+-----------------------+--------+
155166| null| false|
+------+-----------------------+--------+

In [0]:
subcheck_df1 = subcheck_df.withColumn("RN", row_number().over(Window.orderBy(monotonically_increasing_id()))-1)

print(subcheck_df1.select("TSS Methodology", F.col("TSS Methodology").cast('float').isNotNull().alias("IfNumber")).filter("IfNumber == false").count())
subcheck_df1.select("RN", "TSS Methodology", F.col("TSS Methodology").cast('float').isNotNull().alias("IfNumber")).filter("IfNumber == false").show()

155167
+---+--------------------+--------+
 RN| TSS Methodology|IfNumber|
+---+--------------------+--------+
 0|Prior Period Orde...| false|
 1|Order to Quote Ma...| false|
 2| SMT Mapping| false|
 3|Country & SKU Lev...| false|
 4|Order to Quote Ma...| false|
 5| Country Allocation| false|
 6|Prior Period Orde...| false|
 7|Prior Period Orde...| false|
 8| Country Allocation| false|
 9|Country & SKU Lev...| false|
 10|Prior Period Orde...| false|
 11|Prior Period Orde...| false|
 12|SKU Level allocation| false|
 13|Order to Quote Ma...| false|
 14|Country & SKU Lev...| false|
 15|Enriched POS O2Q ...| false|
 16| GSP to BE Mapping| false|
 17| Country Allocation| false|
 18|Order to Quote Ma...| false|
 19| Country Allocation| false|
+---+--------------------+--------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import lit
from  pyspark.sql.functions import input_file_name

subcheck_df = subcheck_df.withColumn("FileName", input_file_name()).withColumn("BE", lit("Collaboration"))

In [0]:
def main(zip_path): 
#     dest_dir = r"/dbfs/mnt/files/ir/files/Bookings/Collaboration/"
    unzip_dir= r"/dbfs/mnt/files/ir/files/Bookings/Test_Unzip"
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.printdir()
            zip_ref.extractall(unzip_dir)
#     for fp in glob.iglob(fr"{dest_dir}/*.csv"):
#         print(f"FROM: {fp}")
#         dest = os.path.join("dbfs:"+unzip_dir[5:],os.path.basename(fp))
#         print(f"TO: {dest}")
#         dbutils.fs.mv("dbfs:"+fp[5:], dest)
    except:
        print("An exception occurred extracting with Python ZipFile library.")
        print("Attempting to extract using 7zip")
        with py7zr.SevenZipFile(zip_path, 'r') as zip_ref:
            zip_ref.printdir()
            zip_ref.extractall(unzip_dir)
#         subprocess.Popen(["7z", "e", f"{zip_path}", f"-o{unzip_dir}", "-y"])

In [0]:
main(source_file_path)

File Name Modified Size
Collaboration_FY16/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201601.csv 2021-09-09 03:55:28 166568929
Collaboration_FY16/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201602.csv 2021-09-09 03:53:24 154634804
Collaboration_FY16/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201603.csv 2021-09-09 03:51:06 245261169
Collaboration_FY16/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201604.csv 2021-09-09 03:48:12 146427511
Collaboration_FY16/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201605.csv 2021-09-09 03:46:22 156271631
Collaboration_FY16/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201606.csv 2021-09-09 03:43:36 282759643
Collaboration_FY16/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201607.csv 2021-09-09 03:38:48 257737771
Collaboration_FY16/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201608.csv 2021-09-09 03:35:56 218286810
Collaboration_FY16/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201609.csv 2021-09-09 03:32:32 280280163
Collaboration_FY16/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201610.csv 2021-09-09 03:28:22 141042645
Collaboration_FY16/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201611.csv 2021-09-09 03:26:18 170461411
Collaboration_FY16/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201612.csv 2021-09-09 03:21:40 345897072

In [0]:
unzip_df = spark.read.format("csv") \
                             .option("header", "true") \
                             .option("delimiter", ",") \
                             .option("inferSchema","true") \
                             .option("ESCAPE quote", '"') \
                             .load('dbfs:/mnt/files/ir/files/Bookings/Test_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201601.csv')

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window
from pyspark.sql import functions as F

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

display(unzip_df.groupBy("Fiscal Period ID").agg({"CA Service Bookings Net": 'sum'}).orderBy(F.col("Fiscal Period ID").asc()).toPandas())

Fiscal Period ID,sum(CA Service Bookings Net)
201601,4.456023126470733E7


In [0]:
union_check1 = unionAll(unzip_df, subcheck_df)
union_check1.groupBy("Fiscal Period ID").agg({"CA Service Bookings Net": 'sum'}).orderBy(F.col("Fiscal Period ID").asc()).toPandas()

Out[42]:

,Fiscal Period ID,sum(CA Service Bookings Net)
0,201601,4.456023e+07
1,201602,5.955642e+07


In [0]:
union_check1.filter(col("Fiscal Period ID") == "201601").select("BE").show()

+----+
 BE|
+----+
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
null|
+----+
only showing top 20 rows

In [0]:
# Set up schema of dataframe
from pyspark.sql.types import StructType

schema = [i for i in unzip_df.schema] 

NewSchema = StructType(schema)

NewSchema

Out[48]: StructType(List(StructField(Fiscal Year,IntegerType,true),StructField(Product ID,StringType,true),StructField(PH-PID,StringType,true),StructField(Fiscal Quarter ID,StringType,true),StructField(Fiscal Period ID,IntegerType,true),StructField(Finance BU/Service Category,StringType,true),StructField(Finance Product Family/Allctd Svcs,StringType,true),StructField(Bookings Transaction Type,StringType,true),StructField(Bill To Global Ultimate Name,StringType,true),StructField(Bill To Global Ultimate ID,IntegerType,true),StructField(ERP Bill To Customer Name,StringType,true),StructField(ERP Bill To Customer Id,IntegerType,true),StructField(ERP Bill To Customer Number,StringType,true),StructField(ERP Sold To Customer Name,StringType,true),StructField(ERP Sold To Customer Id,IntegerType,true),StructField(Ship To Site City,StringType,true),StructField(Ship To Site State,StringType,true),StructField(Ship To Site Country,StringType,true),StructField(ERP Ship To Customer Name,StringType,true),StructField(ERP Ship To Customer Id,IntegerType,true),StructField(End Customer Global Ultimate Name,StringType,true),StructField(ERP End Customer Name,StringType,true),StructField(ERP End Customer Id,IntegerType,true),StructField(ERP Sales Order Number,IntegerType,true),StructField(Web Order ID,StringType,true),StructField(ERP Deal ID,IntegerType,true),StructField(Country,StringType,true),StructField(Sales Level 1,StringType,true),StructField(Sales Level 2,StringType,true),StructField(Sales Level 3,StringType,true),StructField(Sales Level 4,StringType,true),StructField(CA Service Bookings Net,DoubleType,true),StructField(TSS Methodology,StringType,true)))

In [0]:
%fs cp dbfs:/mnt/files/ir/files/Bookings/Test_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201601.csv dbfs:/mnt/files/ir/files/Bookings/Test_Unzip_new/

res12: Boolean = true

In [0]:
%fs cp dbfs:/mnt/files/ir/files/Bookings/Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201602.csv dbfs:/mnt/files/ir/files/Bookings/Test_Unzip_new/

res13: Boolean = true

In [0]:
%fs ls dbfs:/mnt/files/ir/files/Bookings/Test_Unzip_new/

path,name,size
dbfs:/mnt/files/ir/files/Bookings/Test_Unzip_new,Test_Unzip_new,154634804


In [0]:
from pyspark.sql import DataFrame

def Union_all_df(dir):
    import os
    l = os.listdir(dir)
    union_df = spark.createDataFrame([], NewSchema)
#     # Check if union_df exists
#     if union_df is not None and isinstance(union_df,DataFrame):
#       union_df = union_df
#     else:
#       union_df = spark.createDataFrame([], NewSchema)
      
      #  Loop over files in the directory/sub directory
    for d in l:
        if os.path.isdir(dir + d):
            Union_all_df(dir +  d +"/")
        else:
            file_path = dir + d
            print(file_path[5:])
            df = spark.read.format("csv") \
                             .option("header", "true") \
                             .option("delimiter", ",") \
                             .option("inferSchema","true") \
                             .option("ESCAPE quote", '"') \
                             .option("encoding", "UTF-8")\
                             .load(file_path[5:])

            # Union all dfs
            union_df = unionAll(union_df, df)
            print(union_df.count()) # print num of rows unioned
    return union_df


In [0]:
Union_all_df("/dbfs/mnt/files/ir/files/Bookings/Test_Unzip/").groupBy("Fiscal Period ID").agg({"CA Service Bookings Net": 'sum'}).orderBy(F.col("Fiscal Period ID").asc()).toPandas()

--------------------------------------------------------------------------- 
 NotADirectoryError Traceback (most recent call last)
 <command-4471273607937768> in <module> 
 ----> 1 Union_all_df ( r"/dbfs/mnt/files/ir/files/Bookings/Test_Unzip2/" ) . groupBy ( "Fiscal Period ID" ) . agg ( { "CA Service Bookings Net" : 'sum' } ) . orderBy ( F . col ( "Fiscal Period ID" ) . asc ( ) ) . toPandas ( ) 

 <command-4471273607937767> in Union_all_df (dir) 
 3 def Union_all_df ( dir ) : 
 4 import os
 ----> 5 l = os . listdir ( dir ) 
 6 union_df = spark . createDataFrame ( [ ] , NewSchema ) 
 7 # # Check if union_df exists 

 NotADirectoryError : [Errno 20] Not a directory: '/dbfs/mnt/files/ir/files/Bookings/Test_Unzip2/'

In [0]:
%fs rm -r 'dbfs:/mnt/files/ir/files/Bookings/Test_Unzip/TSS_BOOKING_CARMEL_2021Q3_SOonly_pid___Collaboration_201602.csv'

res2: Boolean = true

In [0]:
# Remove input parameter
dbutils.widgets.remove("/dbfs/mnt/files/ir/files/Bookings/Collaboration/Collaboration FY16.zip")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1403433929496203> in <module> 
 1 # Remove input parameter 
 ----> 2 dbutils . widgets . remove ( "/dbfs/mnt/files/ir/files/Bookings/Collaboration/Collaboration FY16.zip" ) 

 /databricks/python_shell/dbruntime/WidgetHandlerImpl.py in remove (self, name) 
 109 : param name : Name of argument associated with input widget to be removed
 110 """
 --> 111 self . _notebookArguments . removeWidget ( name , self . _entry_point . getCurrentBindings ( ) ) 
 112 
 113 def removeAll ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o252.removeWidget.
: com.databricks.dbutils_v1.InputWidgetNotDefined: No input widget named /dbfs/mnt/files/ir/files/Bookings/Collaboration/Collaboration FY16.zip is defined
	at com.databricks.backend.daemon.driver.NotebookArguments.checkExists(NotebookArguments.scala:72)
	at com.databricks.backend.daemon.driver.NotebookArguments.removeWidget(NotebookArguments.scala:229)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)